<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# KNN Classification and Imputation: Cell Phone Churn Data

_Authors: Kiefer Katovich (SF)_

---

In this lab you will practice using KNN for classification (and a little bit for regression as well).

The dataset is one on "churn" in cell phone plans. It has information on the usage of the phones by different account holders and whether or not they churned or not.

Our goal is to predict whether a user will churn or not based on the other features.

We will also be using the KNN model to **impute** missing data. There are a couple of columns in the dataset with missing values, and we can build KNN models to predict what those missing values will most likely be. This is a more advanced imputation method than just filling in the mean or median.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.neighbors import KNeighborsClassifier

### 1. Load the cell phone "churn" data containing some missing values.

In [2]:
churn = pd.read_csv('../data/churn_missing.csv')

### 2. Examine the data. What columns have missing values?

In [8]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
state             3333 non-null object
account_length    3333 non-null int64
area_code         3333 non-null int64
intl_plan         3333 non-null object
vmail_plan        2933 non-null object
vmail_message     2933 non-null float64
day_mins          3333 non-null float64
day_calls         3333 non-null int64
day_charge        3333 non-null float64
eve_mins          3333 non-null float64
eve_calls         3333 non-null int64
eve_charge        3333 non-null float64
night_mins        3333 non-null float64
night_calls       3333 non-null int64
night_charge      3333 non-null float64
intl_mins         3333 non-null float64
intl_calls        3333 non-null int64
intl_charge       3333 non-null float64
custserv_calls    3333 non-null int64
churn             3333 non-null bool
vmail_plan_num    2933 non-null float64
dtypes: bool(1), float64(10), int64(7), object(3)
memory usage: 524.1+ KB

In [3]:
# A: vmail_plan

### 3. Convert the `vmail_plan` and `intl_plan` colums to binary integer columns.

Make sure that if a value is missing that you don't fill it in with a new value! Preserve the missing values.

In [10]:
churn['intl_plan_num'] = churn.intl_plan.map({'yes':1, 'no':0})

In [9]:
churn.intl_plan.unique()

array(['no', 'yes'], dtype=object)

In [4]:
# A:

### 4. Create dummy coded columns for state and concatenate it to the churn dataset.

> **Remember:** You will need to leave out one of the state dummy coded columns to serve as the "reference" column since we will be using these for modeling.

In [12]:
churn_state = pd.get_dummies(churn.state)
churn = pd.concat([churn, churn_state], axis = 1)

In [5]:
# A:

### 5. Create a version of the churn data that has no missing values.

Calculate the shape

In [13]:
churn_new = churn.dropna()

In [15]:
churn_new.shape

(2933, 73)

### 6. Create a target vector and predictor matrix.

- Target should be the `churn` column.
- Predictor matrix should be all columns except `area_code`, `state`, and `churn`.

In [16]:
churn_new.columns

Index([u'state', u'account_length', u'area_code', u'intl_plan', u'vmail_plan',
       u'vmail_message', u'day_mins', u'day_calls', u'day_charge', u'eve_mins',
       u'eve_calls', u'eve_charge', u'night_mins', u'night_calls',
       u'night_charge', u'intl_mins', u'intl_calls', u'intl_charge',
       u'custserv_calls', u'churn', u'vmail_plan_num', u'intl_plan_num', u'AK',
       u'AL', u'AR', u'AZ', u'CA', u'CO', u'CT', u'DC', u'DE', u'FL', u'GA',
       u'HI', u'IA', u'ID', u'IL', u'IN', u'KS', u'KY', u'LA', u'MA', u'MD',
       u'ME', u'MI', u'MN', u'MO', u'MS', u'MT', u'NC', u'ND', u'NE', u'NH',
       u'NJ', u'NM', u'NV', u'NY', u'OH', u'OK', u'OR', u'PA', u'RI', u'SC',
       u'SD', u'TN', u'TX', u'UT', u'VA', u'VT', u'WA', u'WI', u'WV', u'WY'],
      dtype='object')

In [19]:
X = churn.drop('churn', axis = 1)
y = churn.churn

In [24]:
churn

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,KS,128,415,no,yes,25.0,265.1,110,45.07,197.4,...,0,0,0,0,0,0,0,0,0,0
1,OH,107,415,no,yes,26.0,161.6,123,27.47,195.5,...,0,0,0,0,0,0,0,0,0,0
2,NJ,137,415,no,no,0.0,243.4,114,41.38,121.2,...,0,0,0,0,0,0,0,0,0,0
3,OH,84,408,yes,no,0.0,299.4,71,50.90,61.9,...,0,0,0,0,0,0,0,0,0,0
4,OK,75,415,yes,no,0.0,166.7,113,28.34,148.3,...,0,0,0,0,0,0,0,0,0,0
5,AL,118,510,yes,no,0.0,223.4,98,37.98,220.6,...,0,0,0,0,0,0,0,0,0,0
6,MA,121,510,no,NaN,NaN,218.2,88,37.09,348.5,...,0,0,0,0,0,0,0,0,0,0
7,MO,147,415,yes,no,0.0,157.0,79,26.69,103.1,...,0,0,0,0,0,0,0,0,0,0
8,LA,117,408,no,NaN,NaN,184.5,97,31.37,351.6,...,0,0,0,0,0,0,0,0,0,0
9,WV,141,415,yes,yes,37.0,258.6,84,43.96,222.0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
y.sum()/len(y)

0

In [7]:
# A:

### 7. Calculate the baseline accuracy for `churn`.

In [8]:
# A:

### 8. Cross-validate a KNN model predicting `churn`. 

- Number of neighbors should be 5.
- Make sure to standardize the predictor matrix.
- Set cross-validation folds to 10.

Report the mean cross-validated accuracy.

In [9]:
# A:

### 9. Iterate from k=1 to k=49 (only odd k) and cross-validate the accuracy of the model for each.

Plot the cross-validated mean accuracy for each score. What is the best accuracy?

In [10]:
# A:

### 10. Imputing with KNN

K-Nearest Neighbors can be used to impute missing values in datasets. What we will do is estimate the most likely value for the missing data based on a KNN model.

We have two columns with missing data:
- `vmail_plan`
- `vmail_message`

**10.A Create two subsets of the churn dataset: one without missing values for `vmail_plan` and `vmail_message`, and one with the missing values.**

In [11]:
# A:

First we will impute values for `vmail_plan`. This is a categorical column and so we will impute using classification (predicting whether the plan is yes or no, 1 vs. 0).

**10.B Create a target that is `vmail_plan` and predictor matrix that is all columns except `state`, `area_code`, `churn`, `vmail_plan`, and `vmail_message`.**

> **Note:** We don't include the `churn` variable in the model to impute. Why? We are imputing these missing values so that we can use the rows to predict churn with more data afterwards. If we imputed with churn as a predictor then we would be cheating.

In [12]:
# A:

**10.C Standardize the predictor matrix.**

In [13]:
# A:

**10.D Find the best K for predicting `vmail_plan`.**

You may want to write a function for this. What is the accuracy for predicting `vmail_plan` at the best K? What is the baseline accuracy for `vmail_plan`?

In [14]:
# A:

**10.E Fit a `KNeighborsClassifier` with the best number of neighbors.**

In [15]:
# A:

**10.F Predict the missing `vmail_plan` values using the subset of the data where it is misssing.**

You will need to:
1. Create a new predictor matrix using the same predictors but from the missing subset of data.
- Standardize this predictor matrix *using the StandardScaler object fit on the non-missing data*. This means you will just use the `.transform()` function. It is important to standardize the new predictors the same way we standardized the original predictors if we want the predictions to make sense. Calling `.fit_transform()` will reset the standardized scale.
- Predict what the missing vmail plan values should be.
- Replace the missing values in the original with the predicted values.

> **Note:** It may predict all 0's. This is OK. If you want to see the predicted probabilities of `vmail_plan` for each row you can use the `.predict_proba()` function instead of `.predict()`.  You can use these probabilities to manually set the criteria threshold.

In [16]:
# A:

### 11. Impute the missing values for `vmail_message` using the same process.

Since `vmail_message` is essentially a continuous measure, you need to use `KNeighborsRegressor` instead of the `KNeighborsClassifier`.

KNN can do both regression and classification! Instead of "voting" on the class like in classification, the neighbors will average their value for the target in regression.

In [17]:
# A:

### 12. Given the accuracy (and $R^2$) of your best imputation models when finding the best K neighbors, do you think imputing is a good idea?

In [18]:
# A:

### 13. With the imputed dataset, cross-validate the accuracy predicting churn. Is it better? Worse? The same?

In [19]:
# A: